In [1]:
import torch
import wave
import torchaudio
from sympy.codegen.fnodes import dsign
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset

/Library/Python/3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
from datasets import load_dataset, load_metric
from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor, TrainingArguments, Trainer, \
    DataCollatorForSeq2Seq
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")

processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

ds_russian = load_dataset("bond005/sberdevices_golos_10h_crowd", split="validation")
# Preprocessing Russian dataset
def preprocess(example):
    # We don't have target_text for "test".
    target_text = example["translation"] if "translation" in example else ""
    target_text = target_text.lstrip()  # remove leading spaces

    # Prepare speech features
    speech = processor(example["audio"]["array"], sampling_rate=example["audio"]["sampling_rate"], return_tensors="pt")
 
    # Prepare target text
    target = processor.tokenizer(target_text, return_tensors="pt")

    return {
        "input_features": speech.input_features[0],
        "attention_mask": speech.attention_mask[0],
        "labels": target.input_ids[0],
    }

# Preprocess the data
ds_russian = ds_russian.map(preprocess, remove_columns=ds_russian.column_names)
# ds_russian
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./outputs",
    num_train_epochs=10,
    
)

# DataCollator
data_collator = DataCollatorForSeq2Seq(processor, model)
# 
# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_russian,
    data_collator=data_collator        
)
# 
# # Train the model
trainer.train()

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.encoder.embed_positions.weights', 'model.decoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
ds_russian['input_features'][0]

In [ ]:
inputs = processor(ds_russian[0]["audio"]["array"], sampling_rate=ds_russian[0]["audio"]["sampling_rate"], return_tensors="pt")
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
ds_russian[0]["audio"]

In [ ]:
def bytes_to_wav(data):
    output_wav_file = 'output.wav'
    sample_width = 2  # in bytes, e.g., 16-bit audio has a sample width of 2 bytes
    channels = 1  # 1 for mono, 2 for stereo
    sample_rate = 44100  # Samples per second
    with wave.open(output_wav_file, 'wb') as wav_file:
        wav_file.setnchannels(channels)
        wav_file.setsampwidth(sample_width)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(data)

# Example usage:
raw_audio_bytes = ds_russian[0]["audio"]["array"]

bytes_to_wav(raw_audio_bytes)

In [ ]:
ds_russian.features